In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'

In [2]:
import torch
from transformers import pipeline, AutoTokenizer
from datasets import load_from_disk
from vllm.inputs import TokensPrompt
from vllm import LLM, SamplingParams
from tqdm import tqdm



/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# Loading data
data_path= "../datasets/gec/"
split='val'
data=load_from_disk(data_path+split)
# data = load_from_disk(f"{data_path}/tokenized/LLaMA8B-Instruct/{split}/0-shot/")
data

Dataset({
    features: ['id', 'userid', 'cefr', 'text', 'edits'],
    num_rows: 350
})

In [4]:
data[0]

{'id': '1-352420',
 'userid': '21658',
 'cefr': 'A2.ii',
 'text': 'It\'s difficult answer at the question "what are you going to do in the future?" if the only one who has to know it is in two minds. When I was younger I used to say that I wanted to be a teacher, a saleswoman and even a butcher.. I don\'t know why. I would like to study Psychology because one day I would open my own psychology office and help people. It\'s difficult because I\'ll have to study hard and a lot, but I think that if you like a subject, you\'ll study it easier. Maybe I\'ll change my mind, maybe not.',
 'edits': {'start': [14, 21, 150, 228, 280, 304, 328, 466],
  'end': [14, 24, 150, 229, 280, 304, 334, 472],
  'text': [' to', '', ',', '', ',', ' like to', 'clinic', 'more easily']}}

In [5]:
# Loading model
hf_token = os.getenv("hf_token")

model_name= "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id



In [6]:
kwargs={
    'download_dir':cache_dir,
    'max_model_len': 2000
}

model = LLM(
        model=model_name,  
        tensor_parallel_size=1,
        trust_remote_code=True,
    **kwargs
) 



Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-1B-Instruct/.no_exist/9213176726f574b556790deb65791e0c5aa438b6/preprocessor_config.json'


INFO 04-30 19:57:14 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Llama-3.2-1B-Instruct, 

Ignored error while writing commit hash to /scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-1B-Instruct/refs/main: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-1B-Instruct/refs/main'.
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-1B-Instruct/.no_exist/9213176726f574b556790deb65791e0c5aa438b6/model.safetensors.index.json'


INFO 04-30 19:57:17 weight_utils.py:280] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-30 19:57:22 model_runner.py:926] Loading model weights took 2.3185 GB
INFO 04-30 19:57:22 gpu_executor.py:122] # GPU blocks: 73993, # CPU blocks: 8192
INFO 04-30 19:57:24 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-30 19:57:24 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-30 19:57:35 model_runner.py:1335] Graph capturing finished in 11 secs.


In [7]:
stop_strings =  ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>', 'Q:', '</s>', '<|im_end|>']


sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 stop=stop_strings,
                                 # logprobs=2
                                )

In [23]:
import json
task='gec'
task_prompt_path=f'../prompts/{task}.json'
with open(task_prompt_path) as fp:
    task_prompt = json.load(fp)
task_prompt

{'task_prompt': 'Given an input, the goal is to detect and correct the grammatical errors of all error types present in the sentence and return the reasoning first and then the corrected sentence.\n\nYour response should end with "The corrected text is [answer]" where [answer] is the grammatically correct version of the input sentence.',
 'system_msg': 'You are an expert in removing subjective biases in texts.',
 'user_msg': 'Instruction:\n{instruction}\n\nInput:\n{question}\n\n',
 'assistant_msg': '{rationale} The corrected text is {response}\n'}

In [27]:
chat=[
    {
        "role":"system",
        "content":task_prompt['system_msg'],
    },
    {
        "role":"user",
        "content":task_prompt['user_msg'].format(instruction=task_prompt['task_prompt'], question=data[5]['text'])
    }
]
prompt=tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 30 Apr 2025

You are an expert in removing subjective biases in texts.<|eot_id|><|start_header_id|>user<|end_header_id|>

Instruction:
Given an input, the goal is to detect and correct the grammatical errors of all error types present in the sentence and return the reasoning first and then the corrected sentence.

Your response should end with "The corrected text is [answer]" where [answer] is the grammatically correct version of the input sentence.

Input:
Dear Jo,

How are you? I'm writing to inform you that some advice on travelling and working in my country.

On the one hand, in my country there are a lot of place to travel. For example Barcelona,Madrid,Sevilla,Valencia. There are wonderful cities because there are a lot of things to visit for example in Madrid you can visit "Puerta del Sol", "Las Cibeles". I visit Madrid twice a year. All summer I visit Sevilla because I 

In [28]:
all_outputs = model.generate(prompt, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.98s/it, est. speed input: 107.77 toks/s, output: 167.87 toks/s]


In [29]:
print(all_outputs[0].outputs[0].text)

The original sentence contains the following grammatical errors:

1. "inform you" should be "inform you" (subject-verb agreement)
2. "some advice on travelling" should be "some advice on traveling" (subject-verb agreement)
3. "On the one hand" should be "On the one hand" (capitalization)
4. "there are a lot of place to travel" should be "there are a lot of places to travel" (subject-verb agreement)
5. "For example" should be "For example" (capitalization)
6. "Puerta del Sol" and "Las Cibeles" should be "Puerta del Sol" and "Las Cibeles" (capitalization)
7. "I visit Madrid twice a year" should be "I visit Madrid twice a year" (subject-verb agreement)
8. "I love this city" should be "I love this city" (subject-verb agreement)
9. "I suggest that you visit Sevilla" should be "I suggest that you visit Sevilla" (subject-verb agreement)
10. "My best friend looking for a people responsible for look after her sister" should be "My best friend looking for a person responsible for looking after h

In [12]:
data[0]['edits']

{'start': [14, 21, 150, 228, 280, 304, 328, 466],
 'end': [14, 24, 150, 229, 280, 304, 334, 472],
 'text': [' to', '', ',', '', ',', ' like to', 'clinic', 'more easily']}